<a href="https://colab.research.google.com/github/paoins/RMAIA/blob/main/realtime_poller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine
import schedule
import time

In [ ]:
# Create the SQLAlchemy engine
engine = create_engine('sqlite:///rmaia_market_data.db')

In [ ]:
# Fetch the latest market data and append it to the database
def fetch_and_append_data():
  print(f"curent time: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')} ")

  try:

    # Get data for the last day at 5 minute interval
    latest_data = yf.download(
        tickers=['BTC-USD', 'EURUSD=X', 'GLD'],
        period='7d',
        interval='30m',
        auto_adjust=True,
        progress=False,
    )

    # Check if no data is returned = closed market
    if latest_data.empty:
      print("No data returned. Market appears closed.")
      return

    # Flatten the data
    latest_data.columns = ['_'.join(col).strip() for col in latest_data.columns.values]
    latest_data.index.name = 'Date'

    # 3. select only the newest row
    new_row = latest_data.tail(1)

    # write to new
    new_row.to_sql(
        'intraday_realtime',
        engine,
        if_exists='append',
        index=True
        )

    # Quick check
    print(f"SUCCESS: Data was appended. BTC Latest Price: {new_row['Close_BTC-USD'].iloc[0]}")

  except Exception as e:
    print(f"ERROR: {e}")

In [ ]:
fetch_and_append_data()

curent time: 2025-12-10 11:07:40 
SUCCESS: Data was appended. BTC Latest Price: 92321.140625


In [ ]:
# Schecule:

schedule.every(30).minutes.do(fetch_and_append_data)

print('Scheduler is set to fetch and append data every 30 minutes.')
while True:
  schedule.run_pending()
  time.sleep(1)

Scheduler is set to fetch and append data every 30 minutes.
curent time: 2025-12-10 11:07:40 
ERROR: 'BTC-USD_Close'
curent time: 2025-12-10 11:12:40 
ERROR: 'BTC-USD_Close'
